In [ ]:
import requests
from PIL import Image
import gradio as gr
import io

# Custom Vision Prediction 키와 URL
PREDICTION_KEY = "48fBztGiqcoH4PUUK23qmvxTBMmu31mgQ4JmJN1EDeCWNYw2QAtQJQQJ99BEACL93NaXJ3w3AAAIACOGelyc"
ENDPOINT_URL = "https://neptun91cv20250526-prediction.cognitiveservices.azure.com/customvision/v3.0/Prediction/c7c4ea7c-3c10-4d11-9726-dad643e7413e/classify/iterations/Iteration1/image"
headers = {
    "Prediction-Key": PREDICTION_KEY,
    "Content-Type": "application/octet-stream"
}

def predict_with_api(image: Image.Image):
    # 이미지를 바이너리로 변환
    buf = io.BytesIO()
    image.save(buf, format='JPEG')
    byte_data = buf.getvalue()
    
    # API 호출
    response = requests.post(ENDPOINT_URL, headers=headers, data=byte_data)
    predictions = response.json()["predictions"]
    
    # 확률 기준 상위 2개 예측 추출
    top_predictions = sorted(predictions, key=lambda x: x["probability"], reverse=True)[:2]
    
    # 결과 포맷팅
    result_lines = [
        f"{pred['tagName']} ({pred['probability'] * 100:.2f}%)"
        for pred in top_predictions
    ]
    
    return "\n".join(result_lines)

# Gradio 인터페이스 정의
with gr.Blocks() as demo:
    gr.Markdown("## 🧠 Custom Vision Image Classifier")
    gr.Markdown("Upload an image and get predictions from your Azure Custom Vision model.")

    with gr.Row():
        image_input = gr.Image(type="pil", label="Input Image")
        output_text = gr.Textbox(label="Prediction Result", lines=3)

    predict_btn = gr.Button("🔍 Predict")

    predict_btn.click(
        fn=predict_with_api,
        inputs=image_input,
        outputs=output_text
    )

demo.launch()

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.
